In [263]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
import pingouin as pg
import statsmodels.formula.api as smf

# Import data

In [264]:
GameScore = pd.read_csv('GameScore Participants - Blad1.csv', decimal=',')
Demographics = pd.read_csv('MT Demographics_June 13, 2025_02.45.csv', decimal=',')
GameStudy = pd.read_csv('MT Game Study_June 13, 2025_02.45.csv', decimal=',')

# Clean data

### Remove first two rows as they hold no participant data

In [265]:
Demographics = Demographics.drop([0, 1, 46])
GameStudy = GameStudy.drop([0, 1])

### Change column types and data from object to the according type

In [266]:
# Performance
GameScore['Level'] = GameScore['Level'].astype("string")
GameScore['Code'] = pd.to_numeric(GameScore['Code'], errors='coerce')

In [267]:
# Change demographics column to correct column type
Demographics['Q1'] = pd.to_numeric(Demographics['Q1'], errors='coerce')
Demographics['Q2.1'] = pd.to_numeric(Demographics['Q2.1'], errors='coerce')
Demographics['Q3.1'] = Demographics['Q3.1'].astype("string")

# Clean column names
Demographics.columns = Demographics.columns.str.strip().str.replace('\u202f', '', regex=True)

# Define Likert scale mapping
likert_map = {
    'Not apply at all': 1,
    'Rarely apply': 2,
    'Partially apply': 3,
    'Rather apply': 4,
    'Fully apply': 5
}

question_cols = ['Q1.1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10',
                 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19',
                 'Q20', 'Q21', 'Q22', 'Q23']

# Apply Likert scale
for col in question_cols:
    if col in Demographics.columns:
        Demographics[col] = Demographics[col].map(likert_map)

In [268]:
# Game Study
# Fill nan values as it was forgotten to put the question on mandatory. The slider started in the middle at 50
GameStudy['Experience_1'] = GameStudy['Experience_1'].fillna(50)
GameStudy['Experience_2'] = GameStudy['Experience_2'].fillna(50)
GameStudy['Experience_3'] = GameStudy['Experience_3'].fillna(50)
GameStudy['Experience_4'] = GameStudy['Experience_4'].fillna(50)
GameStudy['Experience_5'] = GameStudy['Experience_5'].fillna(50)

# Change column types
GameStudy['Experience_1'] = pd.to_numeric(GameStudy['Experience_1'], errors='coerce')
GameStudy['Experience_2'] = pd.to_numeric(GameStudy['Experience_2'], errors='coerce')
GameStudy['Experience_3'] = pd.to_numeric(GameStudy['Experience_3'], errors='coerce')
GameStudy['Experience_4'] = pd.to_numeric(GameStudy['Experience_4'], errors='coerce')
GameStudy['Experience_5'] = pd.to_numeric(GameStudy['Experience_5'], errors='coerce')
GameStudy['Q1'] = GameStudy['Q1'].astype("string")

GameStudy['Participation Code'] = GameStudy['Participation Code'].astype(str).str.strip()
GameStudy['Participation Code'] = pd.to_numeric(GameStudy['Participation Code'], errors='coerce')

### Remove data of participants that is not usable for the analysis

In [269]:
# Performance
participants_to_remove = [0, 6, 18, 20, 23, 42]
GameScore = GameScore.drop(index=GameScore[GameScore['Code'].isin(participants_to_remove)].index)
GameScore = GameScore.reset_index(drop=True)

In [270]:
# Demographics
participants_to_remove = [0, 6, 18, 20, 23, 42]
Demographics = Demographics.drop(index=Demographics[Demographics['Q1'].isin(participants_to_remove)].index)
Demographics = Demographics.reset_index(drop=True)

In [271]:
# Game Study
participants_to_remove = [0, 6, 18, 20, 23, 42]
GameStudy = GameStudy.drop(index=GameStudy[GameStudy['Participation Code'].isin(participants_to_remove)].index)
GameStudy = GameStudy.reset_index(drop=True)

In [272]:
GameScore = GameScore[GameScore['Level'] != 'Denver'].reset_index(drop=True)
GameStudy = GameStudy[GameStudy['Q1'] != 'Denver'].reset_index(drop=True)

### Split levels in different levels

In [273]:
# Split Performance Levels
PMinneapolis = GameScore[GameScore['Level'] == 'Minneapolis']
PChicago = GameScore[GameScore['Level'] == 'Chicago']

# Split the different sp-hhq factors
Demographics['F1_annoyance'] = Demographics[['Q1.1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8']].mean(axis=1)
Demographics['F2_quality'] = Demographics[['Q9', 'Q10']].mean(axis=1)
Demographics['F3_noise_sensitivity'] = Demographics[['Q11', 'Q12', 'Q13']].mean(axis=1)
Demographics['F4_unpredictable'] = Demographics[['Q14', 'Q15', 'Q16']].mean(axis=1)
Demographics['F5_openness'] = Demographics[['Q17', 'Q18', 'Q19']].mean(axis=1)
Demographics['F6_warm'] = Demographics[['Q20', 'Q21']].mean(axis=1)
Demographics['F7_detail'] = Demographics[['Q22', 'Q23']].mean(axis=1)

Demographics['SPHHQ_Total'] = Demographics[[
    'F1_annoyance', 'F2_quality', 'F3_noise_sensitivity',
    'F4_unpredictable', 'F5_openness', 'F6_warm', 'F7_detail'
]].mean(axis=1)

# Mean of the NASA TLX per participant
GameStudy['Total'] = (GameStudy['Experience_1'] + GameStudy['Experience_2'] + GameStudy['Experience_3'] + GameStudy['Experience_4'] + GameStudy['Experience_5']) / 5

# Split Game Study Levels
CWMinneapolis = GameStudy[GameStudy['Q1'] == 'Minneapolis']
CWChicago = GameStudy[GameStudy['Q1'] == 'Chicago']

### Check for normality

In [274]:
print('Performance Minneapolis: ', shapiro(PMinneapolis['CorrrectMoves']))
print('Performance Chicago: ', shapiro(PChicago['CorrrectMoves']))

print('Cognitive Workload Minneapolis: ',shapiro(CWMinneapolis['Total']))
print('Cognitive Workload Chicago: ',shapiro(CWChicago['Total']))

Performance Minneapolis:  ShapiroResult(statistic=0.8494054079055786, pvalue=0.00010472451685927808)
Performance Chicago:  ShapiroResult(statistic=0.9491802453994751, pvalue=0.07711198925971985)
Cognitive Workload Minneapolis:  ShapiroResult(statistic=0.9848799705505371, pvalue=0.8691685199737549)
Cognitive Workload Chicago:  ShapiroResult(statistic=0.9809184074401855, pvalue=0.7367984056472778)


In [275]:
print('SP-HHQ: ',shapiro(Demographics['SPHHQ_Total']))

SP-HHQ:  ShapiroResult(statistic=0.9228880405426025, pvalue=0.01069800928235054)


In [276]:
# subanalysis H1
subscales = ['Experience_1', 'Experience_2', 'Experience_3', 'Experience_4', 'Experience_5']
for scale in subscales:
    stat, p = stats.shapiro(CWMinneapolis[scale])
    print(f"{scale} (Minneapolis): W={stat:.4f}, p={p:.4f}")

    stat, p = stats.shapiro(CWChicago[scale])
    print(f"{scale} (Chicago): W={stat:.4f}, p={p:.4f}")

Experience_1 (Minneapolis): W=0.9367, p=0.0296
Experience_1 (Chicago): W=0.9534, p=0.1067
Experience_2 (Minneapolis): W=0.9163, p=0.0067
Experience_2 (Chicago): W=0.9764, p=0.5726
Experience_3 (Minneapolis): W=0.9772, p=0.6029
Experience_3 (Chicago): W=0.9534, p=0.1070
Experience_4 (Minneapolis): W=0.9484, p=0.0727
Experience_4 (Chicago): W=0.9435, p=0.0497
Experience_5 (Minneapolis): W=0.9580, p=0.1525
Experience_5 (Chicago): W=0.9773, p=0.6072


In [277]:
# Define SP-HHQ subscales that are used in the analysis
sphhq_subscales = [
    'F1_annoyance', 'F3_noise_sensitivity',
    'F4_unpredictable', 'F5_openness', 'F7_detail'
]

# Loop through conditions
for condition in ['Minneapolis', 'Chicago']:
    print(f'\nShapiro-Wilk test results for {condition} condition:')
    condition_df = df[df['Level'] == condition]
    
    for subscale in sphhq_subscales:
        stat, p = shapiro(condition_df[subscale])
        print(f'{subscale}: W = {stat:.3f}, p = {p:.3f}')



Shapiro-Wilk test results for Minneapolis condition:
F1_annoyance: W = 0.984, p = 0.425
F3_noise_sensitivity: W = 0.965, p = 0.032
F4_unpredictable: W = 0.951, p = 0.005
F5_openness: W = 0.936, p = 0.001
F7_detail: W = 0.949, p = 0.004

Shapiro-Wilk test results for Chicago condition:
F1_annoyance: W = 0.984, p = 0.425
F3_noise_sensitivity: W = 0.965, p = 0.032
F4_unpredictable: W = 0.951, p = 0.005
F5_openness: W = 0.936, p = 0.001
F7_detail: W = 0.949, p = 0.004


### Descriptives

In [278]:
Demographics['Q2.1'].describe()

count    39.000000
mean     31.205128
std      15.096496
min      18.000000
25%      22.000000
50%      24.000000
75%      36.000000
max      72.000000
Name: Q2.1, dtype: float64

In [279]:
Demographics['Q3.1'].value_counts()

Q3.1
Male      20
Female    19
Name: count, dtype: Int64

In [280]:
# List of NASA TLX subscale columns
nasa_subscales = ['Experience_1', 'Experience_2', 'Experience_3', 
                  'Experience_4', 'Experience_5']

# Group by condition and describe each subscale
descriptives = GameStudy.groupby('Q1')[nasa_subscales].describe()
descriptives

Experience_1                                                      \
                   count       mean        std   min   25%   50%   75%   max   
Q1                                                                             
Chicago             39.0  55.512821  20.862800  10.0  40.0  60.0  70.0  90.0   
Minneapolis         39.0  52.564103  18.878831  10.0  40.0  60.0  65.0  85.0   

            Experience_2             ... Experience_4       Experience_5  \
                   count       mean  ...          75%   max        count   
Q1                                   ...                                   
Chicago             39.0  56.794872  ...         72.5  90.0         39.0   
Minneapolis         39.0  51.410256  ...         70.0  95.0         39.0   

                                                                 
                  mean        std  min   25%   50%   75%    max  
Q1                                                               
Chicago      56.923077  24.132730  0.0  40.0  60.0  75.0  100.0  
Minneapolis  48.076923  26.695539  0.0  27.5  55.0  70.0  100.0  

[2 rows x 40 columns]

# Data Analysis

#### H1: On-beat music leads to lower cognitive workload and higher task performance than off-beat music.

In [281]:
print('Cognitive workload; ', stats.wilcoxon(CWMinneapolis['Total'], CWChicago['Total']))
print('Performance: ', stats.wilcoxon(PMinneapolis['CorrrectMoves'], PChicago['CorrrectMoves']))

Cognitive workload;  WilcoxonResult(statistic=230.0, pvalue=0.1054120574709588)
Performance:  WilcoxonResult(statistic=277.0, pvalue=0.1172088714374695)


/Users/mijketaal/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [282]:
subscales = ['Experience_1', 'Experience_2', 'Experience_3', 'Experience_4', 'Experience_5']
for scale in subscales:
    stat, p = stats.wilcoxon(CWMinneapolis[scale], CWChicago[scale])
    print(f"{scale}: W={stat}, p={p:.3f}")

Experience_1: W=326.5, p=0.705
Experience_2: W=224.5, p=0.136
Experience_3: W=198.5, p=0.142
Experience_4: W=273.0, p=0.490
Experience_5: W=143.0, p=0.008


In [283]:
print("Mean frustration (on-beat):", CWMinneapolis['Experience_5'].mean())
print("Mean frustration (off-beat):", CWChicago['Experience_5'].mean())

Mean frustration (on-beat): 48.07692307692308
Mean frustration (off-beat): 56.92307692307692


#### H3: Players with higher audio susceptibility exhibit greater performance improvements and lower cognitive workload when playing with rhythmic music.

In [284]:
# Merge GameScore and GameStudy on participant code
merged_scores_study = GameScore.merge(
    GameStudy,
    left_on='Code',
    right_on='Participation Code',
    how='inner'
)

# Merge the result with Demographics
full_merged = merged_scores_study.merge(
    Demographics,
    left_on='Code',
    right_on='Q1',
    how='inner'
)

In [285]:
# Average performance and workload per participant across conditions
aggregated = full_merged.groupby('Code').agg({
    'CorrrectMoves': 'mean',
    'Total': 'mean',
    'SPHHQ_Total': 'first'
}).reset_index()

In [286]:
# Correlation: SPHHQ vs Performance
perf_corr, perf_p = spearmanr(aggregated['SPHHQ_Total'], aggregated['CorrrectMoves'])
print(f"SPHHQ vs Performance: r = {perf_corr:.2f}, p = {perf_p:.4f}")

# Correlation: SPHHQ vs Cognitive Load
load_corr, load_p = spearmanr(aggregated['SPHHQ_Total'], aggregated['Total'])
print(f"SPHHQ vs Cognitive Load: r = {load_corr:.2f}, p = {load_p:.4f}")

SPHHQ vs Performance: r = -0.22, p = 0.1754
SPHHQ vs Cognitive Load: r = 0.29, p = 0.0728


In [287]:
# Filter for on-beat (Minneapolis) and off-beat (Chicago) conditions
on_beat = full_merged[full_merged['Level'] == 'Minneapolis']
off_beat = full_merged[full_merged['Level'] == 'Chicago']

# SPHHQ vs Performance (CorrectMoves)
print("SPPHW vs Performance (on beat): ",spearmanr(on_beat['SPHHQ_Total'], on_beat['CorrrectMoves']))
print("SPPHW vs Performance (off beat): ",spearmanr(off_beat['SPHHQ_Total'], off_beat['CorrrectMoves']))
# SPHHQ vs Cognitive Load (Total)
print("SPPHW vs CL (on beat)", spearmanr(on_beat['SPHHQ_Total'], on_beat['Total']))
print("SPPHW vs CL (off beat)", spearmanr(off_beat['SPHHQ_Total'], off_beat['Total']))

SPPHW vs Performance (on beat):  SignificanceResult(statistic=-0.013261793953358908, pvalue=0.9082556554910253)
SPPHW vs Performance (off beat):  SignificanceResult(statistic=-0.18132118660102411, pvalue=0.11212119653867315)
SPPHW vs CL (on beat) SignificanceResult(statistic=0.29301519630091427, pvalue=0.009228498116820501)
SPPHW vs CL (off beat) SignificanceResult(statistic=0.29301519630091427, pvalue=0.009228498116820501)


In [288]:
data = full_merged.dropna(subset=['Total','SPHHQ_Total','Level']).copy()

data['Level'] = data['Level'].astype('category')               
data['SPHHQ_Total'] = pd.to_numeric(data['SPHHQ_Total'])      
data['Total']       = pd.to_numeric(data['Total'])

model = smf.ols('Total ~ SPHHQ_Total * Level', data=data).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                  Total   R-squared:                       0.093
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     5.184
Date:                Tue, 01 Jul 2025   Prob (F-statistic):            0.00195
Time:                        23:21:59   Log-Likelihood:                -614.28
No. Observations:                 156   AIC:                             1237.
Df Residuals:                     152   BIC:                             1249.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

In [289]:
# Replace with your actual dataframe name
df = full_merged  # or whatever your merged dataframe is called

# Define your subscale column names (update if your column names are different)
sphhq_subscales = ['F1_annoyance', 'F3_noise_sensitivity',
    'F4_unpredictable', 'F5_openness', 'F7_detail']

# Loop through conditions
for condition in ['Minneapolis', 'Chicago']:
    print(f'\n{condition} condition:')
    condition_df = df[df['Level'] == condition]
    
    for subscale in sphhq_subscales:
        # Correlate with cognitive workload (NASA TLX total)
        r_cw, p_cw = spearmanr(condition_df[subscale], condition_df['Total'])
        # Correlate with performance
        r_perf, p_perf = spearmanr(condition_df[subscale], condition_df['CorrrectMoves'])

        print(f'{subscale} vs Cognitive Workload: r = {r_cw:.2f}, p = {p_cw:.3f}')
        print(f'{subscale} vs Performance:       r = {r_perf:.2f}, p = {p_perf:.3f}')


Minneapolis condition:
F1_annoyance vs Cognitive Workload: r = 0.13, p = 0.253
F1_annoyance vs Performance:       r = -0.00, p = 0.982
F3_noise_sensitivity vs Cognitive Workload: r = 0.05, p = 0.688
F3_noise_sensitivity vs Performance:       r = -0.17, p = 0.147
F4_unpredictable vs Cognitive Workload: r = 0.11, p = 0.355
F4_unpredictable vs Performance:       r = 0.11, p = 0.338
F5_openness vs Cognitive Workload: r = 0.29, p = 0.010
F5_openness vs Performance:       r = 0.19, p = 0.089
F7_detail vs Cognitive Workload: r = 0.22, p = 0.050
F7_detail vs Performance:       r = 0.03, p = 0.780

Chicago condition:
F1_annoyance vs Cognitive Workload: r = 0.13, p = 0.253
F1_annoyance vs Performance:       r = 0.14, p = 0.212
F3_noise_sensitivity vs Cognitive Workload: r = 0.05, p = 0.688
F3_noise_sensitivity vs Performance:       r = 0.14, p = 0.223
F4_unpredictable vs Cognitive Workload: r = 0.11, p = 0.355
F4_unpredictable vs Performance:       r = -0.12, p = 0.306
F5_openness vs Cognitive 

## Exploratory Analysis

In [290]:
# Define mapping
mapping = {
    "Strongly disagree": 1,
    "Somewhat disagree": 2,
    "Neither agree nor disagree": 3,
    "Somewhat agree": 4,
    "Strongly agree": 5
}

Demographics["Q5.1"] = Demographics["Q5.1"].map(mapping)

In [291]:
# Define mapping
mapping = {
    "Very poor": 1,
    "Poor": 2,
    "Average": 3,
    "Good": 4,
    "Very Good": 5
}

Demographics["Q6.1"] = Demographics["Q6.1"].map(mapping)

# Demographics['Q6.1']

In [292]:
# Define mapping
mapping = {
    "Never": 1,
    "Rarely": 2,
    "Sometimes": 3,
    "Often": 4,
    "Always": 5
}

Demographics["Q7.1"] = Demographics["Q7.1"].map(mapping)

### Check normality

In [293]:
print('Q5.1', shapiro(Demographics['Q5.1']))
print('Q6.1', shapiro(Demographics['Q6.1']))
print('Q6.1', shapiro(Demographics['Q7.1']))

Q5.1 ShapiroResult(statistic=0.8248322606086731, pvalue=2.8494516300270334e-05)
Q6.1 ShapiroResult(statistic=0.9021820425987244, pvalue=0.0025496180169284344)
Q6.1 ShapiroResult(statistic=0.88898766040802, pvalue=0.0010826460784301162)


### Exploratory correlations

In [294]:
print('Minnapolis vs Q5.1: ', spearmanr(PMinneapolis['CorrrectMoves'],Demographics['Q5.1']))
print('Chicago vs Q5.1: ',spearmanr(PChicago['CorrrectMoves'],Demographics['Q5.1']))
print('Chicago vs Q6.1: ',spearmanr(PChicago['CorrrectMoves'],Demographics['Q6.1']))
print('Minnapolis vs Q6.1: ', spearmanr(PMinneapolis['CorrrectMoves'],Demographics['Q6.1']))
print('Minnapolis vs Q7.1: ',spearmanr(CWMinneapolis['Total'],Demographics['Q7.1']))
print('Minnapolis vs Q7.1: ', spearmanr(CWChicago['Total'],Demographics['Q7.1']))

Minnapolis vs Q5.1:  SignificanceResult(statistic=0.17995738239796874, pvalue=0.27296991136066434)
Chicago vs Q5.1:  SignificanceResult(statistic=0.12226328569734327, pvalue=0.4584020993250498)
Chicago vs Q6.1:  SignificanceResult(statistic=0.057006846148856836, pvalue=0.7303159474268215)
Minnapolis vs Q6.1:  SignificanceResult(statistic=0.17306276120133388, pvalue=0.29207122083954346)
Minnapolis vs Q7.1:  SignificanceResult(statistic=0.13112919426238615, pvalue=0.4262047728389816)
Minnapolis vs Q7.1:  SignificanceResult(statistic=0.1945746944265595, pvalue=0.23523961447604574)


In [295]:
df = pd.DataFrame({
    'Subject': Demographics['Q1'],  
    'Minneapolis': PMinneapolis['CorrrectMoves'],
    'Chicago': PChicago['CorrrectMoves'],
    'MusicalTraining': Demographics['Q5.1']  
})

df['TrainingGroup'] = df['MusicalTraining'].apply(lambda x: 'low' if x <= 3 else 'high')

df_long = pd.melt(df, 
                  id_vars=['Subject', 'TrainingGroup'], 
                  value_vars=['Minneapolis', 'Chicago'], 
                  var_name='Condition', 
                  value_name='CorrectMoves')


In [296]:
# Join on participant ID to ensure proper alignment
df = pd.DataFrame()
df['Subject'] = Demographics['Q1']
df['MusicalTraining'] = Demographics['Q5.1']

# Merge performance from both conditions
df = df.merge(PMinneapolis[['Code', 'CorrrectMoves']], how='left', left_on='Subject', right_on='Code')
df = df.rename(columns={'CorrrectMoves': 'Minneapolis'})
df = df.drop(columns='Code')

df = df.merge(PChicago[['Code', 'CorrrectMoves']], how='left', left_on='Subject', right_on='Code')
df = df.rename(columns={'CorrrectMoves': 'Chicago'})
df = df.drop(columns='Code')

# Drop rows with any missing values
df = df.dropna(subset=['Minneapolis', 'Chicago', 'MusicalTraining'])

# Binarize training
df['TrainingGroup'] = df['MusicalTraining'].apply(lambda x: 'low' if x <= 3 else 'high')


In [297]:
df_long = pd.melt(df,
                  id_vars=['Subject', 'TrainingGroup'],
                  value_vars=['Minneapolis', 'Chicago'],
                  var_name='Condition',
                  value_name='CorrectMoves')

#A mixed‐design ANOVA on CorrectMoves per condition as the within‐subjects factor and the musical training as the between‐subjects factor
aov = pg.mixed_anova(dv='CorrectMoves',
                     within='Condition',
                     between='TrainingGroup',
                     subject='Subject',
                     data=df_long)

print(aov)


          Source          SS  DF1  DF2          MS         F     p-unc  \
0  TrainingGroup    3.817211    1   37    3.817211  0.026674  0.871155   
1      Condition  291.160128    1   37  291.160128  2.771510  0.104403   
2    Interaction    5.980848    1   37    5.980848  0.056931  0.812731   

        np2  eps  
0  0.000720  NaN  
1  0.069686  1.0  
2  0.001536  NaN  
